In [1]:
import torch
from torch.utils.data import Dataset
from torch.utils.data.dataloader import DataLoader

In [3]:
#Data must be accesible from that directory

import pandas as pd
data = []

with open('hpc/train_GPT', 'r') as f:
    for line in f:
        value = line.strip()
        data.append(value)

# Convert to DataFrame
sys20_full = pd.DataFrame(data, columns=["EventId"])

data = []

with open('bgl/train_GPT', 'r') as f:
    for line in f:
        value = line.strip()
        data.append(value)

# Convert to DataFrame
bgl_df = pd.DataFrame(data, columns=["EventId"])

In [4]:
def processBGL(bgl_df):
  bgl_df["Abnormal"] = bgl_df["Label"].apply(lambda x: int(x != "-"))
  bgl_df['Abnormal-EventId'] = bgl_df['Abnormal'].astype(str) + "-" + bgl_df['EventId']
  vocab = {id: idx + 1 for idx, id in enumerate(bgl_df['Abnormal-EventId'].unique())}
  vocab['<PAD>'] = 0
  reverse_vocab = {index: label for label, index in vocab.items()}
  bgl_df['Tokenized'] = bgl_df['Abnormal-EventId'].map(vocab)

  # Check for tokens starting with '1'
  error_tokens = [vocab[token] for token in vocab.keys() if token.startswith('1')]

  return bgl_df, vocab, reverse_vocab, error_tokens


In [5]:
def processBGL(bgl_df):
  vocab = {id: idx + 1 for idx, id in enumerate(bgl_df['EventId'].unique())}
  vocab['<PAD>'] = 0
  reverse_vocab = {index: label for label, index in vocab.items()}
  bgl_df['Tokenized'] = bgl_df['EventId'].map(vocab)

  # Check for tokens starting with '1'
  error_tokens = [vocab[token] for token in vocab.keys() if token.startswith('1')]

  return bgl_df, vocab, reverse_vocab, error_tokens

In [6]:
import pandas as pd

def processHPC(df):
  error_events = [
    'e7ede03f',
    'e07f96ea',
    '3dff51fb',
    'bd82b62d',
    '562e2cee',
    'b02b9741',
    '74e8333a',
    '252f035f',
    '683ade8b',
    '7e85bcb6',
    '5cdfd2f3',
    '7b80ea70',
    '0cc39f34',
    'c0fcff27',
    'a84ecf37',
    '20bdc066',
    'b38fa951',
    '8c1777d1',
    'dfc857c8',
    '6eff2f7f',
    '03b9852a'
    ]

  # Assuming df['EventId'] is already defined in your DataFrame 'df'
  # Start the enumeration from 1, reserving 0 for padding
  vocab = {id: idx + 1 for idx, id in enumerate(df['EventId'].unique())}

  # Adding padding token to the vocabulary
  vocab['<PAD>'] = 0

  # Creating a reverse vocabulary
  reverse_vocab = {index: label for label, index in vocab.items()}

  # Mapping the EventId to their respective indices in the vocabulary
  df['Tokenized'] = df['EventId'].map(vocab)

  error_tokens = [vocab[event] for event in error_events if event in vocab]

  return df, vocab, reverse_vocab, error_tokens

In [7]:
#df, vocab, reverse_vocab, error_tokens = processBGL(bgl_df)
df, vocab, reverse_vocab, error_tokens = processHPC(sys20_full)

In [19]:
import math
import torch
import torch.nn as nn
from torch.nn import functional as F

import numpy as np
import random
import os
import json
import sys
from ast import literal_eval

"""
MinGPT code
Source: https://github.com/karpathy/minGPT/blob/master/mingpt
"""

class CfgNode:
    """ a lightweight configuration class inspired by yacs """
    # TODO: convert to subclass from a dict like in yacs?
    # TODO: implement freezing to prevent shooting of own foot
    # TODO: additional existence/override checks when reading/writing params?

    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

    def __str__(self):
        return self._str_helper(0)

    def _str_helper(self, indent):
        """ need to have a helper to support nested indentation for pretty printing """
        parts = []
        for k, v in self.__dict__.items():
            if isinstance(v, CfgNode):
                parts.append("%s:\n" % k)
                parts.append(v._str_helper(indent + 1))
            else:
                parts.append("%s: %s\n" % (k, v))
        parts = [' ' * (indent * 4) + p for p in parts]
        return "".join(parts)

    def to_dict(self):
        """ return a dict representation of the config """
        return { k: v.to_dict() if isinstance(v, CfgNode) else v for k, v in self.__dict__.items() }

    def merge_from_dict(self, d):
        self.__dict__.update(d)

    def merge_from_args(self, args):
        """
        update the configuration from a list of strings that is expected
        to come from the command line, i.e. sys.argv[1:].

        The arguments are expected to be in the form of `--arg=value`, and
        the arg can use . to denote nested sub-attributes. Example:

        --model.n_layer=10 --trainer.batch_size=32
        """
        for arg in args:

            keyval = arg.split('=')
            assert len(keyval) == 2, "expecting each override arg to be of form --arg=value, got %s" % arg
            key, val = keyval # unpack

            # first translate val into a python object
            try:
                val = literal_eval(val)
                """
                need some explanation here.
                - if val is simply a string, literal_eval will throw a ValueError
                - if val represents a thing (like an 3, 3.14, [1,2,3], False, None, etc.) it will get created
                """
            except ValueError:
                pass

            # find the appropriate object to insert the attribute into
            assert key[:2] == '--'
            key = key[2:] # strip the '--'
            keys = key.split('.')
            obj = self
            for k in keys[:-1]:
                obj = getattr(obj, k)
            leaf_key = keys[-1]

            # ensure that this attribute exists
            assert hasattr(obj, leaf_key), f"{key} is not an attribute that exists in the config"

            # overwrite the attribute
            print("command line overwriting config attribute %s with %s" % (key, val))
            setattr(obj, leaf_key, val)

class NewGELU(nn.Module):
    """
    Implementation of the GELU activation function currently in Google BERT repo (identical to OpenAI GPT).
    Reference: Gaussian Error Linear Units (GELU) paper: https://arxiv.org/abs/1606.08415
    """
    def forward(self, x):
        return 0.5 * x * (1.0 + torch.tanh(math.sqrt(2.0 / math.pi) * (x + 0.044715 * torch.pow(x, 3.0))))



class CausalSelfAttention(nn.Module):
    """
    A vanilla multi-head masked self-attention layer with a projection at the end.
    It is possible to use torch.nn.MultiheadAttention here but I am including an
    explicit implementation here to show that there is nothing too scary here.
    """

    def __init__(self, config):
        super().__init__()
        assert config.n_embd % config.n_head == 0
        # key, query, value projections for all heads, but in a batch
        self.c_attn = nn.Linear(config.n_embd, 3 * config.n_embd)
        # output projection
        self.c_proj = nn.Linear(config.n_embd, config.n_embd)
        # regularization
        self.attn_dropout = nn.Dropout(config.attn_pdrop)
        self.resid_dropout = nn.Dropout(config.resid_pdrop)
        # causal mask to ensure that attention is only applied to the left in the input sequence
        self.register_buffer("bias", torch.tril(torch.ones(config.block_size, config.block_size))
                                     .view(1, 1, config.block_size, config.block_size))
        self.n_head = config.n_head
        self.n_embd = config.n_embd

    def forward(self, x):
        B, T, C = x.size() # batch size, sequence length, embedding dimensionality (n_embd)

        # calculate query, key, values for all heads in batch and move head forward to be the batch dim
        q, k ,v  = self.c_attn(x).split(self.n_embd, dim=2)
        k = k.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        q = q.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        v = v.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)

        # causal self-attention; Self-attend: (B, nh, T, hs) x (B, nh, hs, T) -> (B, nh, T, T)
        att = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(k.size(-1)))
        att = att.masked_fill(self.bias[:,:,:T,:T] == 0, float('-inf'))
        att = F.softmax(att, dim=-1)
        att = self.attn_dropout(att)
        y = att @ v # (B, nh, T, T) x (B, nh, T, hs) -> (B, nh, T, hs)
        y = y.transpose(1, 2).contiguous().view(B, T, C) # re-assemble all head outputs side by side

        # output projection
        y = self.resid_dropout(self.c_proj(y))
        return y

'''
#Additive Attention
class CausalSelfAttention(nn.Module):
    """
    A vanilla multi-head masked self-attention layer with a projection at the end.
    It is possible to use torch.nn.MultiheadAttention here but I am including an
    explicit implementation here to show that there is nothing too scary here.
    """

    def __init__(self, config):
        super().__init__()
        assert config.n_embd % config.n_head == 0
        # key, query, value projections for all heads, but in a batch
        self.c_attn = nn.Linear(config.n_embd, 3 * config.n_embd)
        # output projection
        self.c_proj = nn.Linear(config.n_embd, config.n_embd)
        # regularization
        self.attn_dropout = nn.Dropout(config.attn_pdrop)
        self.resid_dropout = nn.Dropout(config.resid_pdrop)
        # causal mask to ensure that attention is only applied to the left in the input sequence
        self.register_buffer("bias", torch.tril(torch.ones(config.block_size, config.block_size))
                                     .view(1, 1, config.block_size, config.block_size))

        # Additive attention parameters
        self.W1 = nn.Linear(config.n_embd // config.n_head, config.n_embd // config.n_head, bias=False)
        self.W2 = nn.Linear(config.n_embd // config.n_head, config.n_embd // config.n_head, bias=False)
        self.V = nn.Linear(config.n_embd // config.n_head, 1, bias=False)

        self.n_head = config.n_head
        self.n_embd = config.n_embd

    def forward(self, x):
        B, T, C = x.size() # batch size, sequence length, embedding dimensionality (n_embd)

        # calculate query, key, values for all heads in batch and move head forward to be the batch dim
        q, k ,v  = self.c_attn(x).split(self.n_embd, dim=2)
        k = k.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        q = q.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        v = v.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)

        # causal self-attention; Self-attend: (B, nh, T, hs) x (B, nh, hs, T) -> (B, nh, T, T)
        # att = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(k.size(-1)))

        # additive attention score computation
        q_proj = self.W1(q)
        k_proj = self.W2(k)
        att = self.V(torch.tanh(q_proj.unsqueeze(-2) + k_proj.unsqueeze(-3))).squeeze(-1)

        att = att.masked_fill(self.bias[:,:,:T,:T] == 0, float('-inf'))
        att = F.softmax(att, dim=-1)
        att = self.attn_dropout(att)
        y = att @ v # (B, nh, T, T) x (B, nh, T, hs) -> (B, nh, T, hs)
        y = y.transpose(1, 2).contiguous().view(B, T, C) # re-assemble all head outputs side by side

        # output projection
        y = self.resid_dropout(self.c_proj(y))
        return y
'''

class HierarchicalAttention(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.word_attention = CausalSelfAttention(config)
        self.sentence_attention = CausalSelfAttention(config)
        self.ln = nn.LayerNorm(config.n_embd)

    def forward(self, x):
        # Applying word-level attention
        word_level_output = self.word_attention(x)
        word_level_output = self.ln(word_level_output)

        # Reshaping for sentence-level attention
        B, T, C = word_level_output.size()
        sentence_input = word_level_output.view(B, -1, T, C).mean(dim=2)

        # Applying sentence-level attention
        sentence_level_output = self.sentence_attention(sentence_input)

        # Combining outputs
        combined_output = word_level_output + sentence_level_output.unsqueeze(2).repeat(1, 1, T, 1).view(B, T, C)
        return combined_output

#For Causal Self Attention
class Block(nn.Module):
    """ an unassuming Transformer block """

    def __init__(self, config):
        super().__init__()
        self.ln_1 = nn.LayerNorm(config.n_embd)
        self.attn = CausalSelfAttention(config)
        self.ln_2 = nn.LayerNorm(config.n_embd)

        # Two-layer feed-forward network with ReLU activation
        self.ffn = nn.Sequential(
            nn.Linear(config.n_embd, 4 * config.n_embd),
            nn.ReLU(),
            nn.Linear(4 * config.n_embd, config.n_embd),
        )
        self.dropout = nn.Dropout(config.resid_pdrop)

    def forward(self, x):
        x = x + self.attn(self.ln_1(x))
        # x = x + self.mlpf(self.ln_2(x))
        x = x + self.dropout(self.ffn(self.ln_2(x)))
        return x

'''
#For Hierarchical attention
class Block(nn.Module):
    """ A Transformer block with hierarchical attention """

    def __init__(self, config):
        super().__init__()
        self.ln_1 = nn.LayerNorm(config.n_embd)
        self.attn = HierarchicalAttention(config)
        self.ln_2 = nn.LayerNorm(config.n_embd)
        self.ffn = nn.Sequential(
            nn.Linear(config.n_embd, 4 * config.n_embd),
            nn.ReLU(),
            nn.Linear(4 * config.n_embd, config.n_embd),
        )
        self.dropout = nn.Dropout(config.resid_pdrop)

    def forward(self, x):
        x = x + self.attn(self.ln_1(x))
        x = x + self.dropout(self.ffn(self.ln_2(x)))
        return x
'''

class PositionalEncoding(nn.Module):
  """
  Uses sine and cosine functions to create positional encodings, which will be added to the log event embeddings
  Params:
  - d_model: dimension size
  - max_len: block size
  """
  def __init__(self, d_model, max_len):
      super(PositionalEncoding, self).__init__()
      pe = torch.zeros(max_len, d_model)
      # matrix of position indices (max_len, 1)
      position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
      # computing positional encodings
      # div_term = torch.pow(10000, torch.arange(0, d_model, 2, dtype=torch.float, device=device) / d_model)
      div_term = (torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model)).exp()
      pe[:, 0::2] = torch.sin(position * div_term) # even positions in input vectors of log events in sequence
      pe[:, 1::2] = torch.cos(position * div_term) # odd positions in input vectors of log events in sequence
      pe = pe.unsqueeze(0)
      self.register_buffer('pe', pe)

  def forward(self, x):
      return self.pe[:, :x.size(1)]

class GPT(nn.Module):
    """ GPT Language Model """

    @staticmethod
    def get_default_config():
        C = CfgNode()
        # either model_type or (n_layer, n_head, n_embd) must be given in the config
        C.model_type = 'gpt'
        C.n_layer = None
        C.n_head = None
        C.n_embd =  None
        # these options must be filled in externally
        C.vocab_size = None
        C.block_size = None
        # dropout hyperparameters
        C.embd_pdrop = 0.1
        C.resid_pdrop = 0.1
        C.attn_pdrop = 0.1
        return C

    def __init__(self, config):
        super().__init__()
        assert config.vocab_size is not None
        assert config.block_size is not None
        assert config.input_size is not None
        assert config.target_size is not None


        self.block_size = config.block_size
        self.input_size = config.input_size
        self.target_size = config.target_size

        type_given = config.model_type is not None
        params_given = all([config.n_layer is not None, config.n_head is not None, config.n_embd is not None])
        assert type_given ^ params_given # exactly one of these (XOR)
        if type_given:
            # translate from model_type to detailed configuration
            config.merge_from_dict({
                # names follow the huggingface naming conventions
                # GPT-1
                'openai-gpt':   dict(n_layer=12, n_head=12, n_embd=768),  # 117M params
                # GPT-2 configs
                'gpt2':         dict(n_layer=12, n_head=12, n_embd=768),  # 124M params
                'gpt2-medium':  dict(n_layer=24, n_head=16, n_embd=1024), # 350M params
                'gpt2-large':   dict(n_layer=36, n_head=20, n_embd=1280), # 774M params
                'gpt2-xl':      dict(n_layer=48, n_head=25, n_embd=1600), # 1558M params
                # Gophers
                'gopher-44m':   dict(n_layer=8, n_head=16, n_embd=512),
                # (there are a number more...)
                # I made these tiny models up
                'gpt-mini':     dict(n_layer=6, n_head=6, n_embd=192),
                'gpt-micro':    dict(n_layer=4, n_head=4, n_embd=128),
                'gpt-nano':     dict(n_layer=3, n_head=3, n_embd=48),
            }[config.model_type])

        self.transformer = nn.ModuleDict(dict(
            wte = nn.Embedding(config.vocab_size, config.n_embd),
            wpe = PositionalEncoding(config.n_embd, config.block_size),
            drop = nn.Dropout(config.embd_pdrop),
            h = nn.ModuleList([Block(config) for _ in range(config.n_layer)]),
            ln_f = nn.LayerNorm(config.n_embd),
        ))
        self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias=False)

        # init all weights, and apply a special scaled init to the residual projections, per GPT-2 paper
        self.apply(self._init_weights)
        for pn, p in self.named_parameters():
            if pn.endswith('c_proj.weight'):
                torch.nn.init.normal_(p, mean=0.0, std=0.02/math.sqrt(2 * config.n_layer))

        # report number of parameters (note we don't count the decoder parameters in lm_head)
        n_params = sum(p.numel() for p in self.transformer.parameters())
        print("number of parameters: %.2fM" % (n_params/1e6,))

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
        elif isinstance(module, nn.LayerNorm):
            torch.nn.init.zeros_(module.bias)
            torch.nn.init.ones_(module.weight)

    def from_pretrained(self, model):
        self.block_size = model.block_size
        self.input_size = model.input_size
        self.target_size = model.target_size

        self.transformer = model.transformer
        self.lm_head = model.lm_head

    
    def configure_optimizers(self, train_config):
        """
        This long function is unfortunately doing something very simple and is being very defensive:
        We are separating out all parameters of the model into two buckets: those that will experience
        weight decay for regularization and those that won't (biases, and layernorm/embedding weights).
        We are then returning the PyTorch optimizer object.
        """

        # separate out all parameters to those that will and won't experience regularizing weight decay
        decay = set()
        no_decay = set()
        whitelist_weight_modules = (torch.nn.Linear, )
        blacklist_weight_modules = (torch.nn.LayerNorm, torch.nn.Embedding)
        for mn, m in self.named_modules():
            for pn, p in m.named_parameters():
                fpn = '%s.%s' % (mn, pn) if mn else pn # full param name
                # random note: because named_modules and named_parameters are recursive
                # we will see the same tensors p many many times. but doing it this way
                # allows us to know which parent module any tensor p belongs to...
                if pn.endswith('bias'):
                    # all biases will not be decayed
                    no_decay.add(fpn)
                elif pn.endswith('weight') and isinstance(m, whitelist_weight_modules):
                    # weights of whitelist modules will be weight decayed
                    decay.add(fpn)
                elif pn.endswith('weight') and isinstance(m, blacklist_weight_modules):
                    # weights of blacklist modules will NOT be weight decayed
                    no_decay.add(fpn)

        # validate that we considered every parameter
        param_dict = {pn: p for pn, p in self.named_parameters()}
        inter_params = decay & no_decay
        union_params = decay | no_decay
        assert len(inter_params) == 0, "parameters %s made it into both decay/no_decay sets!" % (str(inter_params), )
        assert len(param_dict.keys() - union_params) == 0, "parameters %s were not separated into either decay/no_decay set!" \
                                                    % (str(param_dict.keys() - union_params), )

        # create the pytorch optimizer object
        optim_groups = [
            {"params": [param_dict[pn] for pn in sorted(list(decay))], "weight_decay": train_config.weight_decay},
            {"params": [param_dict[pn] for pn in sorted(list(no_decay))], "weight_decay": 0.0},
        ]
        optimizer = torch.optim.AdamW(optim_groups, lr=train_config.learning_rate, betas=train_config.betas)
        return optimizer

    def forward(self, idx, targets=None, use_softmax = False):
        device = idx.device
        if use_softmax:
            b, t, _ = idx.size()
        else:
            b, t = idx.size()
        assert t <= self.block_size, f"Cannot forward sequence of length {t}, block size is only {self.block_size}"

        # pos = torch.arange(0, t, dtype=torch.long, device=device).unsqueeze(0) # shape (1, t)
        pos = torch.arange(0, t, dtype=torch.long, device=device).unsqueeze(0).unsqueeze(-1)  # shape (1, t, 1)

        if use_softmax:
            # Assuming idx is the softmax distribution here
            tok_emb = torch.matmul(idx, self.transformer.wte.weight)  # token embeddings of shape (b, t, n_embd)
        else:
            tok_emb = self.transformer.wte(idx)  # token embeddings of shape (b, t, n_embd)
        pos_emb = self.transformer.wpe(pos) # position embeddings of shape (1, t, n_embd)
        x = self.transformer.drop(tok_emb + pos_emb)
        for block in self.transformer.h:
            x = block(x)
        x = self.transformer.ln_f(x)
        logits = self.lm_head(x)

        # if we are given some desired targets also calculate the loss
        loss = None
        if targets is not None:
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1), ignore_index=-1)

        return logits, loss

    @torch.no_grad()
    def generate(self, idx, max_new_tokens, temperature=1.0, do_sample=False, top_k=None):
        """
        Take a conditioning sequence of indices idx (LongTensor of shape (b,t)) and complete
        the sequence max_new_tokens times, feeding the predictions back into the model each time.
        Most likely you'll want to make sure to be in model.eval() mode of operation for this.
        """
        for _ in range(max_new_tokens):
            # if the sequence context is growing too long we must crop it at block_size
            idx_cond = idx if idx.size(1) <= self.block_size else idx[:, -self.block_size:]
            # forward the model to get the logits for the index in the sequence
            logits, _ = self(idx_cond)
            # pluck the logits at the final step and scale by desired temperature
            logits = logits[:, -1, :] / temperature
            # optionally crop the logits to only the top k options
            if top_k is not None:
                v, _ = torch.topk(logits, top_k)
                logits[logits < v[:, [-1]]] = -float('Inf')
            # apply softmax to convert logits to (normalized) probabilities
            probs = F.softmax(logits, dim=-1)
            # either sample from the distribution or take the most likely element
            if do_sample:
                idx_next = torch.multinomial(probs, num_samples=1)
            else:
                _, idx_next = torch.topk(probs, k=1, dim=-1)
            # append sampled index to the running sequence and continue
            idx = torch.cat((idx, idx_next), dim=1)

        return idx

    def predict_next_event(self, idx, max_new_tokens, sequence_length=1, temperature=1.0, use_softmax= False):
      """
      Predict the next event given a sequence of indices idx.
      """
      for _ in range(max_new_tokens):
          # forward pass to get the logits
          logits, _ = self(idx, use_softmax = use_softmax)
          # applying softmax to get probabilities
          probs = F.softmax(logits[:, -sequence_length, :] / temperature, dim=-1)
          next_event_idx = probs
          if not use_softmax:
              # index of the event with the highest probability
              next_event_idx = torch.argmax(probs, dim=-1)
          # appending the predicted event index to the input sequence
          idx = torch.cat((idx, next_event_idx.unsqueeze(1)), dim=1)
          # removing the first element to maintain a constant sequence length
          idx = idx[:, 1:]

      return idx

    def predict_log_sequence(self, idx, sequence_length, vocabulary, predict_next_sequence_length = 1, use_softmax= False):
        """
        Predict a log sequence of the desired length.
        """

        if predict_next_sequence_length <= 1:
          temperature = 1.0

          logits, _ = self(idx)

          last_logits = logits[:, -sequence_length:, :]

          probs = F.softmax(last_logits / temperature, dim=-1)

          next_event_idx = torch.argmax(probs, dim=-1)

          return next_event_idx

        else:
          predicted_sequence = idx.clone()
          if use_softmax:
            vocabulary_size = len(vocabulary)

            # Create a one-hot encoding tensor
            b, t = predicted_sequence.shape
            prob = torch.zeros(b, t, vocabulary_size, device=predicted_sequence.device)
            prob.scatter_(2, predicted_sequence.unsqueeze(-1), 1)

            predicted_sequence = prob

          

          for i in range(predict_next_sequence_length):
            #print(i, ": ",predicted_sequence[0])
            # predicting the next event
            predicted_sequence = self.predict_next_event(predicted_sequence, 1, use_softmax = use_softmax)

          actaul_predictions = predicted_sequence[:, -predict_next_sequence_length * sequence_length:]

          return predicted_sequence



In [9]:
#GPT Dataset preperation
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from tokenizers import ByteLevelBPETokenizer
import numpy as np

class EventLogDataset(Dataset):
    def __init__(self, dataframe, input_size, target_size, block_size, split = "train"):
        assert block_size >= input_size
        assert block_size >= target_size

        self.split = split
        self.block_size = block_size
        self.target_size = target_size
        self.input_size = input_size

        # Ensure each item in the dataframe['Tokenized'] is a tensor of fixed length
        self.full_data = dataframe['Tokenized'].values

    def __len__(self):
        if self.split == "train" :
          return len(self.full_data) - self.block_size
        else:
          return len(self.full_data) - self.input_size - self.target_size

    def __getitem__(self, idx):
        arr = np.array(self.full_data)
        cat = None
        if self.split == "train" :
          cat = arr[idx:idx+self.input_size + 1]
        else:
          cat = arr[idx:idx+self.input_size + self.target_size]

        input_seq = None
        input_seq = torch.tensor(cat[:self.block_size], dtype=torch.long).clone()
        target_seq = torch.tensor(cat[-self.block_size:], dtype=torch.long).clone()

        # We only want to predict at output locations, mask out the loss at the input locations
        if self.split == "test":
          input_seq[self.input_size:] = 0
        if self.split == "train":
          target_seq[:- 1] = -1

        return input_seq, target_seq  # Return the input and target sequences

In [10]:
"""
Simple training loop; Boilerplate that could apply to any arbitrary neural network,
so nothing in this file really has anything to do with GPT specifically.
"""
from tqdm import tqdm
import itertools

import time
from collections import defaultdict

import torch
from torch.utils.data.dataloader import DataLoader

class Trainer:

    @staticmethod
    def get_default_config():
        C = CfgNode()
        # device to train on
        C.device = 'auto'
        # dataloder parameters
        C.num_workers = 4
        # optimizer parameters
        C.max_iters = None
        C.batch_size = 64
        C.learning_rate = 3e-4
        C.betas = (0.9, 0.95)
        C.weight_decay = 0.1 # only applied on matmul weights
        C.grad_norm_clip = 1.0
        C.num_epochs = 10
        C.validation_interval = 500
        return C

    def __init__(self, config, model, train_dataset, val_dataset):
        self.config = config
        self.model = model
        self.optimizer = None
        self.predictions = None
        self.y = None
        self.train_dataset = train_dataset
        self.val_dataset = val_dataset
        self.callbacks = defaultdict(list)
        self.num_epochs = config.num_epochs

        # determine the device we'll train on
        if config.device == 'auto':
            self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        else:
            self.device = config.device
        self.model = self.model.to(self.device)
        print("running on device", self.device)

        # variables that will be assigned to trainer class later for logging and etc
        self.iter_num = 0
        self.iter_time = 0.0
        self.iter_dt = 0.0

    def add_callback(self, onevent: str, callback):
        self.callbacks[onevent].append(callback)

    def set_callback(self, onevent: str, callback):
        self.callbacks[onevent] = [callback]

    def trigger_callbacks(self, onevent: str):
        for callback in self.callbacks.get(onevent, []):
            callback(self)

    def evaluate(self, val_loader):
        self.model.eval()
        total_correct = 0
        total_loss = 0
        total_examples = 0

        with torch.no_grad():
            for x, y in val_loader:
                x, y = x.to(self.device), y.to(self.device)
                logits, loss = self.model(x, y)

                predictions = torch.argmax(logits, dim=-1)

                # Mask to ignore -1 values
                mask = y != -1

                # Apply the mask to the predictions and the true labels
                masked_predictions = predictions[:, -1]
                masked_labels = y[:, -1]

                # Calculate the number of correct predictions
                correct = (masked_predictions == masked_labels).sum().item()

                total_correct += correct
                total_loss += loss.item() * x.size(0)
                total_examples += x.size(0)

        avg_loss = total_loss / total_examples
        avg_accuracy = total_correct / total_examples

        return avg_loss, avg_accuracy

    def run(self):
        model, config = self.model, self.config

        # setup the optimizer
        self.optimizer = model.configure_optimizers(config)

        # setup the dataloader
        train_loader = DataLoader(
            self.train_dataset,
            shuffle=True,
            pin_memory=True,
            batch_size=config.batch_size,
            num_workers=config.num_workers,
        )

        val_loader = DataLoader(
            self.val_dataset,
            shuffle=False,
            pin_memory=True,
            batch_size=config.batch_size,
            num_workers=config.num_workers,
        )

        model.train()
        self.iter_num = 0
        self.iter_time = time.time()
        losses = []


        for epoch in range(self.num_epochs):
            start_time = time.time()
            total_correct = 0
            total_examples = 0
            total_loss = 0
            # training

            # Let's say you want to use only the first 100 batches from the train_loader
            num_batches = self.config.max_iters
            
            # Use itertools.islice to slice the iterator
            sliced_train_loader = itertools.islice(train_loader, num_batches)
            
            # Enumerate the sliced iterator
            progress_bar = tqdm(enumerate(sliced_train_loader), total=num_batches, desc=f'Epoch {epoch + 1}/{self.num_epochs}')

            for i, batch in progress_bar:
              x, y = batch
              x, y = x.to(self.device), y.to(self.device)


              logits, self.loss = model(x, y)
              model.zero_grad(set_to_none=True)
              self.loss.backward()
              torch.nn.utils.clip_grad_norm_(model.parameters(), config.grad_norm_clip)
              self.optimizer.step()


              self.iter_num += 1

              predictions = torch.argmax(logits, dim=-1)

              # Mask to ignore -1 values
              mask = y != -1

                # Apply the mask to the predictions and the true labels
              masked_predictions = predictions[:,-1]
              masked_labels = y[:,-1]

              # Calculate the number of correct predictions
              correct = (masked_predictions == masked_labels).sum().item()


              total_correct += correct
              total_examples += x.size(0)

              if i % 100 == 0:
                progress_bar.set_postfix(loss=self.loss.item(), accuracy=float(correct) / x.size(0))

              total_loss += self.loss.item() * x.size(0)

              if self.iter_num % config.validation_interval == 0:
                train_loss = total_loss / total_examples
                train_accuracy = total_correct / total_examples

                val_loss, val_accuracy = self.evaluate(val_loader)

                print(f"Step {self.iter_num}: Train Loss = {train_loss:.4f}, Train Accuracy = {train_accuracy:.4f}, "
                      f"Val Loss = {val_loss:.4f}, Val Accuracy = {val_accuracy:.4f}")
                model.train()


            train_loss = total_loss / total_examples
            losses.append(train_loss)
            train_accuracy = total_correct / total_examples

            epoch_time = time.time() - start_time

            #variables for reporting
            self.epoch = epoch
            self.train_loss = train_loss
            self.train_accuracy = train_accuracy
            self.epoch_time = epoch_time

            self.trigger_callbacks('on_batch_end')



In [11]:
def filter_adjacent_values(df, block_size):
    filtered_rows = []
    current_value = None
    count = 0

    for index, row in df.iterrows():
        value = row["Tokenized"]
        if value == current_value:
            count += 1
        else:
            current_value = value
            count = 1
        
        if count <= block_size + 1:
            filtered_rows.append(row)
    
    return pd.DataFrame(filtered_rows)

In [12]:
class EventLogDataset(Dataset):
    def __init__(self, dataframes, input_size, target_size, block_size, split="train"):
        assert block_size >= input_size
        assert block_size >= target_size

        self.split = split
        self.block_size = block_size
        self.target_size = target_size
        self.input_size = input_size

        # Store the dataframes
        self.dataframes = dataframes
        self.full_data = [df['Tokenized'].values for df in dataframes]

    def __len__(self):
        total_length = 0
        for data in self.full_data:
            if self.split == "train":
                total_length += len(data) - self.block_size
            else:
                total_length += len(data) - self.input_size - self.target_size
        return total_length

    def __getitem__(self, idx):
        # Determine which dataframe the idx falls into
        current_idx = idx
        for data in self.full_data:
            length = len(data) - (self.block_size if self.split == "train" else self.input_size + self.target_size)
            if current_idx < length:
                arr = np.array(data)
                cat = None
                if self.split == "train":
                    cat = arr[current_idx:current_idx+self.input_size + 1]
                else:
                    cat = arr[current_idx:current_idx+self.input_size + self.target_size]

                input_seq = torch.tensor(cat[:self.block_size], dtype=torch.long).clone()
                target_seq = torch.tensor(cat[-self.block_size:], dtype=torch.long).clone()

                # We only want to predict at output locations, mask out the loss at the input locations
                if self.split == "test":
                    input_seq[self.input_size:] = 0
                if self.split == "train":
                    target_seq[:-1] = -1

                return input_seq, target_seq  # Return the input and target sequences
            else:
                current_idx -= length

        raise IndexError("Index out of range")

In [13]:
#Data Preperation

import time
import torch
import torch.optim as optim

GPTconfig = GPT.get_default_config();
GPTconfig.model_type = 'gpt2'
GPTconfig.vocab_size = len(vocab)
GPTconfig.input_size = 512 # number of log events in a sequence
GPTconfig.target_size = 512
GPTconfig.block_size = 512

filtered_train_df = filter_adjacent_values(df, GPTconfig.block_size)

np.random.seed(55)

# Assuming 'df' is your DataFrame containing the tokenized data
val_size = int(len(filtered_train_df) * 0.01)
validation_indices = np.random.choice(len(filtered_train_df) - val_size, 10, replace=False)
validation_dfs = [filtered_train_df.iloc[idx:idx + val_size] for idx in validation_indices]

# Get the training data excluding validation indices
train_dfs = []
start_idx = 0
sorted_validation_indices = sorted(validation_indices)

for idx in sorted(validation_indices):
    if start_idx < idx:
        train_dfs.append(filtered_train_df.iloc[start_idx:idx])
    start_idx = idx + val_size

# Append the remaining part after the last validation index
if start_idx < len(filtered_train_df):
    train_dfs.append(filtered_train_df.iloc[start_idx:])


train_dataset = EventLogDataset(train_dfs, GPTconfig.input_size, GPTconfig.target_size, GPTconfig.block_size)
train_validation_dataset = EventLogDataset(validation_dfs, GPTconfig.input_size, GPTconfig.target_size, GPTconfig.block_size)

validation_dataset = EventLogDataset(validation_dfs, GPTconfig.input_size, GPTconfig.target_size, GPTconfig.block_size, split="test")
print('training set:', len(train_dataset), 'validation set:', len(validation_dataset))

#model initialization
model = GPT(GPTconfig)

training set: 260436 validation set: 19140
number of parameters: 113.49M


In [119]:
model.from_pretrained(model_new)

In [118]:
import torch

model_new = torch.load('bgl_train.pth')

In [14]:
#Trainer Initialization

train_config = Trainer.get_default_config()
train_config.learning_rate = 0.0001 # the model we're using is so small that we can go a bit faster
train_config.max_iters = 2500
train_config.num_workers = 0
train_config.batch_size = 32
train_config.num_epochs = 10 #200
train_config.validation_interval = 100
trainer = Trainer(train_config, model, train_dataset, train_validation_dataset)

running on device cuda


In [15]:
def batch_end_callback(trainer):
    print(f'Epoch [{trainer.epoch+1}/{trainer.num_epochs}], Train Loss: {trainer.train_loss:.4f}, Train Acc: {trainer.train_accuracy:.4f}, Time: {trainer.epoch_time:.2f} sec')
trainer.set_callback('on_batch_end', batch_end_callback)
trainer.run()

C:\Users\hozgu\anaconda3\envs\nanoGPT\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Epoch 1/10: 100%|██████████| 2500/2500 [17:42<00:00,  2.35it/s, accuracy=0.531, loss=0.995]


Epoch [1/10], Train Loss: 0.9547, Train Acc: 0.6327, Time: 1062.94 sec


Epoch 2/10: 100%|██████████| 2500/2500 [17:35<00:00,  2.37it/s, accuracy=0.812, loss=0.564]


Epoch [2/10], Train Loss: 0.6658, Train Acc: 0.7650, Time: 1055.24 sec


Epoch 3/10: 100%|██████████| 2500/2500 [17:28<00:00,  2.38it/s, accuracy=0.75, loss=0.598] 


...

Epoch [4/10], Train Loss: 0.5922, Train Acc: 0.7835, Time: 1046.77 sec


Epoch 5/10: 100%|██████████| 2500/2500 [17:28<00:00,  2.39it/s, accuracy=0.75, loss=0.73]  


Epoch [5/10], Train Loss: 0.5874, Train Acc: 0.7858, Time: 1048.08 sec


Epoch 6/10:   0%|          | 7/2500 [00:03<18:46,  2.21it/s, accuracy=0.875, loss=0.536]


KeyboardInterrupt: 

In [16]:
from sklearn.metrics import precision_recall_fscore_support

def evaluate_predictions(model, data_loader, vocabulary, trainer, target_size, unique_error_ids, use_softmax=False):
    correct_predictions = 0
    total_predictions = 0
    true_labels = []
    predicted_labels = []

    # For error predictions
    true_errors = []
    predicted_errors = []

    # For error occurrence predictions
    true_error_occurrence = []
    predicted_error_occurrence = []

    model.eval()

    with torch.no_grad():
        pbar = tqdm(data_loader, desc="Evaluating")
        for b, (input_sequence, target_sequence) in enumerate(pbar):
            # Move model to CPU
            #model.to('cpu')

            # Change the device in your trainer configuration to CPU
            #trainer.device = torch.device('cpu')

            # predictions for the input sequence
            input_sequence, target_sequence = input_sequence.to(trainer.device), target_sequence.to(trainer.device)
            predicted_sequence = model.predict_log_sequence(input_sequence, 1, vocabulary, target_size, use_softmax=use_softmax)

            if use_softmax:
              # index of the event with the highest probability
              predicted_sequence = torch.argmax(predicted_sequence, dim=-1)


            predicted_labels.extend(predicted_sequence)  # Assuming sequence might already be a list or similar

            target_sequence = target_sequence[:, -target_size:]
            true_labels.extend(target_sequence)

            # Check each token if it is an error
            batch_true_errors = [1 if label.item() in unique_error_ids else 0 for seq in target_sequence for label in seq.flatten()]
            batch_predicted_errors = [1 if label.item() in unique_error_ids else 0 for seq in predicted_sequence for label in seq.flatten()]

            true_errors.extend(batch_true_errors)
            predicted_errors.extend(batch_predicted_errors)

            # Determine if an error occurs in each sequence
            batch_true_occurrence = [1 if any(label.item() in unique_error_ids for label in seq) else 0 for seq in target_sequence]
            batch_predicted_occurrence = [1 if any(label.item() in unique_error_ids for label in seq) else 0 for seq in predicted_sequence]

            true_error_occurrence.extend(batch_true_occurrence)
            predicted_error_occurrence.extend(batch_predicted_occurrence)

            # Calculate intermediate accuracies
            current_true_labels = [token.item() for batch in true_labels for token in batch.flatten()]
            current_predicted_labels = [token.item() for batch in predicted_labels for token in batch.flatten()]

            current_true_errors = true_errors
            current_predicted_errors = predicted_errors

            current_true_occurrence = true_error_occurrence
            current_predicted_occurrence = predicted_error_occurrence

            # General accuracy
            accuracy = sum(1 for true, pred in zip(current_true_labels, current_predicted_labels) if true == pred) / len(current_true_labels)

            # Error predictions accuracy
            error_accuracy = sum(1 for true, pred in zip(current_true_errors, current_predicted_errors) if true == pred) / len(current_true_errors)

            # Error occurrence predictions accuracy
            occurrence_accuracy = sum(1 for true, pred in zip(current_true_occurrence, current_predicted_occurrence) if true == pred) / len(current_true_occurrence)

            # Update progress bar with intermediate accuracies
            pbar.set_description(f"Evaluating (Acc: {accuracy:.4f}, Err Acc: {error_accuracy:.4f}, Occ Acc: {occurrence_accuracy:.4f})")

            if (b + 1) % 30 == 0:


                break

    true_labels = [token.item() for batch in true_labels for token in batch.flatten()]
    #true_labels = [vocabulary[label if label != -1 else 0] for label in true_labels]

    predicted_labels = [token.item() for batch in predicted_labels for token in batch.flatten()]
    #true_labels = [vocabulary[token.item()] for token in true_labels]
    #print(predicted_labels[0], "   ", true_labels[0])


    #true_errors = [token for batch in true_errors for token in batch]
    #predicted_errors = [token for batch in predicted_errors for token in batch]

    precision, recall, f1, _ = precision_recall_fscore_support(true_labels, predicted_labels, average='macro')
    accuracy = sum(1 for true, pred in zip(true_labels, predicted_labels) if true == pred) / len(true_labels)

        # Evaluation for error predictions
    error_precision, error_recall, error_f1, _ = precision_recall_fscore_support(true_errors, predicted_errors, average='binary')
    error_accuracy = sum(1 for true, pred in zip(true_errors, predicted_errors) if true == pred) / len(true_errors)

    # Evaluation for error occurrence predictions
    occurrence_precision, occurrence_recall, occurrence_f1, _ = precision_recall_fscore_support(true_error_occurrence, predicted_error_occurrence, average='binary')
    occurrence_accuracy = sum(1 for true, pred in zip(true_error_occurrence, predicted_error_occurrence) if true == pred) / len(true_error_occurrence)

    return {
        'accuracy': accuracy, 'precision': precision, 'recall': recall, 'f1': f1,
        'error_accuracy': error_accuracy, 'error_precision': error_precision, 'error_recall': error_recall, 'error_f1': error_f1,
        'occurrence_accuracy': occurrence_accuracy, 'occurrence_precision': occurrence_precision, 'occurrence_recall': occurrence_recall, 'occurrence_f1': occurrence_f1
    }

In [18]:


validation_loader = DataLoader(validation_dataset, batch_size=train_config.batch_size, pin_memory=True, shuffle=True, drop_last=False, num_workers=train_config.num_workers,)
# Evaluate the predictions (use_softmax = true for probabilistic inference)
evaluation_results = evaluate_predictions(model, validation_loader, reverse_vocab, trainer, validation_dataset.target_size, error_tokens, use_softmax = False)

# Extract metrics for validation
validation_accuracy = evaluation_results['accuracy']
validation_precision = evaluation_results['precision']
validation_recall = evaluation_results['recall']
validation_f1 = evaluation_results['f1']

# Additional metrics for error analysis might be of interest
error_accuracy = evaluation_results['error_accuracy']
error_precision = evaluation_results['error_precision']
error_recall = evaluation_results['error_recall']
error_f1 = evaluation_results['error_f1']

occurrence_accuracy = evaluation_results['occurrence_accuracy']
occurrence_precision = evaluation_results['occurrence_precision']
occurrence_recall = evaluation_results['occurrence_recall']
occurrence_f1 = evaluation_results['occurrence_f1']

# Print the standard validation metrics
print(f'Validation Accuracy: {validation_accuracy:.4f}, Validation Precision: {validation_precision:.4f}, Recall: {validation_recall:.4f}, F1: {validation_f1:.4f}')

# Optionally print the error prediction metrics
print(f'Error Prediction - Accuracy: {error_accuracy:.4f}, Precision: {error_precision:.4f}, Recall: {error_recall:.4f}, F1: {error_f1:.4f}')

# Optionally print the error occurrence metrics
print(f'Error Occurrence - Accuracy: {occurrence_accuracy:.4f}, Precision: {occurrence_precision:.4f}, Recall: {occurrence_recall:.4f}, F1: {occurrence_f1:.4f}')

Evaluating (Acc: 0.5273, Err Acc: 0.9006, Occ Acc: 0.4562):   5%|▍         | 29/599 [52:08<17:04:58, 107.89s/it]


Validation Accuracy: 0.5273, Validation Precision: 0.0966, Recall: 0.0949, F1: 0.0944
Error Prediction - Accuracy: 0.9006, Precision: 0.2498, Recall: 0.2128, F1: 0.2298
Error Occurrence - Accuracy: 0.4562, Precision: 1.0000, Recall: 0.0984, F1: 0.1792


C:\Users\hozgu\anaconda3\envs\nanoGPT\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\hozgu\anaconda3\envs\nanoGPT\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
